# Time performance foxsisim before reflecivity speed up

Using the **cProfile** python module (suggested by Albert Shi) we were able to test the performance of the program and find out the reason the inclusion of the energy response was taking so much time of the simulation. After some close look of the problem we found the the function responsable for 98% of the procesing time was the *griddata* function in *reflectivity.py*. Wroking with Steven Christe, we changed the function to **inter2d** that not only will work faster but also will give the right reflectivity of the foxsi optics (we found a bug in the old version). <br /> 

Today, July 11, 2017, Steven merged the foxsisim repository with the new version of the reflectivity. The idea is to run this notebook as well as *Time-testing-reflectivity-after-speed-up* to check the inprovement in time.

## Including Energy response for 1000 rays

In [2]:
import cProfile
from foxsisim.module import Module
from foxsisim.detector import Detector
from foxsisim.source import Source
from foxsisim.plotting import plot,scatterHist
import matplotlib.pyplot as plt
import numpy as np

max_energy = 30.0
nrays = 1000

def spectrum(z):
        if (type(z) is not type([1])) and (type(z) is not type(np.array(1))):
            x = np.array([z])
        else:
            x = np.array(z)
        return np.piecewise(x, [x < 0, ((x < max_energy) & (x > 0)), (x >= max_energy)], [0, 1./max_energy, 0])

source_distance = -1e4 ##cm
source = Source(type='point', center=[0, 0, source_distance])
source.loadSpectrum(spectrum)
energies = np.arange(-10, 60, 0.1)

module = Module(radii=[5.151,4.9],focal=200.0)
detector = Detector(width=8,
                    height=8,
                    normal = [0,0,1],
                    center = [0,0,230],
                    reso = [1024,1024])

In [3]:
cProfile.run('rays = source.generateRays(module.targetFront, nrays)')

         55060236 function calls (55060232 primitive calls) in 69.692 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  2007453   15.820    0.000   62.082    0.000 <ipython-input-2-9ae8287834ed>:12(spectrum)
        1    0.000    0.000   69.692   69.692 <string>:1(<module>)
        3    0.000    0.000    0.000    0.000 <string>:12(__new__)
        1    0.000    0.000    0.000    0.000 <string>:2(<module>)
     8421    0.002    0.000    0.002    0.000 <string>:2(_parse_args)
        1    0.000    0.000    0.000    0.000 <string>:5(_parse_args_rvs)
        1    0.000    0.000    0.001    0.001 _distn_infrastructure.py:1484(__init__)
     8421    0.026    0.000   69.637    0.008 _distn_infrastructure.py:1565(_ppf_to_solve)
     1000    0.011    0.000   69.665    0.070 _distn_infrastructure.py:1568(_ppf_single)
     8421    0.327    0.000   69.610    0.008 _distn_infrastructure.py:1706(cdf)
        3    0.000    0.000    0.000 

In [4]:
cProfile.run('module.passRays(rays, robust=True)')

[ 0.  0.  0.]
0 ray bounce number 1
[-5.22344354 -1.06378599  1.60356627]
x =  None
0 ray killed by reflect
[array([     0.,      0., -10000.]), array([-5.22344354, -1.06378599,  1.60356627])]
[ 0.  0.  0.]
[ 0.  0.  0.]
1 ray bounce number 1
[-4.54056614 -2.75174951  5.02851684]
x =  None
1 ray killed by reflect
[array([     0.,      0., -10000.]), array([-4.54056614, -2.75174951,  5.02851684])]
[ 0.  0.  0.]
[ 0.  0.  0.]
2 ray bounce number 1
[-5.15576604  1.23976662  6.0828502 ]
x =  [ 0.01267457 -0.00304775  0.99991503]
2 ray bounce number 2
[ -4.5349976    1.0904953   55.05616489]
x =  None
2 ray killed by reflect
[array([     0.,      0., -10000.]), array([-5.15576604,  1.23976662,  6.0828502 ]), array([ -4.5349976 ,   1.0904953 ,  55.05616489])]
[ 0.  0.  0.]
[ 0.  0.  0.]
3 ray bounce number 1
[  4.17888595   3.08058667  23.662364  ]
x =  None
3 ray killed by reflect
[array([     0.,      0., -10000.]), array([  4.17888595,   3.08058667,  23.662364  ])]
[ 0.  0.  0.]
[ 0.  0. 

x =  [-0.00524788 -0.01222724  0.99991147]
34 ray bounce number 2
[  1.98470602   4.62424396  36.14458856]
x =  [-0.01002471 -0.02335696  0.99967693]
34 ray bounce number 3
[  1.98470602   4.62424396  36.14458856]
x =  None
34 ray killed by reflect
[array([     0.,      0., -10000.]), array([  2.04518651,   4.7651598 ,  24.62086581]), array([  1.98470602,   4.62424396,  36.14458856]), array([  1.98470602,   4.62424396,  36.14458856])]
[ 0.  0.  0.]
[ 0.  0.  0.]
35 ray bounce number 1
[ 4.57367456  2.72291829  2.85842442]
x =  [-0.01116234 -0.00664545  0.99991562]
35 ray bounce number 2
[  3.95776176   2.35623715  58.03152808]
x =  [-0.02248135 -0.01338418  0.99965767]
35 ray bounce number 3
[  3.95776176   2.35623715  58.03152808]
x =  None
35 ray killed by reflect
[array([     0.,      0., -10000.]), array([ 4.57367456,  2.72291829,  2.85842442]), array([  3.95776176,   2.35623715,  58.03152808]), array([  3.95776176,   2.35623715,  58.03152808])]
[ 0.  0.  0.]
[ 0.  0.  0.]
36 ray b

x =  None
65 ray killed by reflect
[array([     0.,      0., -10000.]), array([  3.73929408,   3.51189582,  31.09291452])]
[ 0.  0.  0.]
[ 0.  0.  0.]
66 ray bounce number 1
[ 1.62986887  5.03199071  8.21712929]
x =  None
66 ray killed by reflect
[array([     0.,      0., -10000.]), array([ 1.62986887,  5.03199071,  8.21712929])]
[ 0.  0.  0.]
[ 0.  0.  0.]
67 ray bounce number 1
[  4.40839235   2.88344595  11.67348282]
x =  None
67 ray killed by reflect
[array([     0.,      0., -10000.]), array([  4.40839235,   2.88344595,  11.67348282])]
[ 0.  0.  0.]
[ 0.  0.  0.]
68 ray bounce number 1
[ 4.28066397 -3.12839098  6.20394078]
x =  None
68 ray killed by reflect
[array([     0.,      0., -10000.]), array([ 4.28066397, -3.12839098,  6.20394078])]
[ 0.  0.  0.]
[ 0.  0.  0.]
69 ray bounce number 1
[ -2.73553036   4.42096599  22.53198668]
x =  None
69 ray killed by reflect
[array([     0.,      0., -10000.]), array([ -2.73553036,   4.42096599,  22.53198668])]
[ 0.  0.  0.]
[ 0.  0.  0.]
7

x =  None
102 ray killed by reflect
[array([     0.,      0., -10000.]), array([-4.60840831,  2.67336986,  2.0811048 ])]
[ 0.  0.  0.]
[ 0.  0.  0.]
103 ray bounce number 1
[-4.80142023  2.2220099   8.01264199]
x =  [ 0.01185515 -0.00548635  0.99991467]
103 ray bounce number 2
[ -4.26526408   1.97388659  53.23437123]
x =  [ 0.02358382 -0.01091416  0.99966229]
[array([     0.,      0., -10000.]), array([-4.80142023,  2.2220099 ,  8.01264199]), array([ -4.26526408,   1.97388659,  53.23437123])]
[ 0.  0.  0.]
[ 0.  0.  0.]
104 ray bounce number 1
[  1.20321655  -5.08311575  18.64573922]
x =  None
104 ray killed by reflect
[array([     0.,      0., -10000.]), array([  1.20321655,  -5.08311575,  18.64573922])]
[ 0.  0.  0.]
[ 0.  0.  0.]
105 ray bounce number 1
[ -3.89777798   3.53505892  12.56115625]
x =  None
105 ray killed by reflect
[array([     0.,      0., -10000.]), array([ -3.89777798,   3.53505892,  12.56115625])]
[ 0.  0.  0.]
[ 0.  0.  0.]
106 ray bounce number 1
[  1.26473228   

x =  None
135 ray killed by reflect
[array([     0.,      0., -10000.]), array([  4.96521273,   1.41517976,  28.14123197])]
[ 0.  0.  0.]
[ 0.  0.  0.]
136 ray bounce number 1
[ -4.62943511  -2.42164114  18.49164167]
x =  None
136 ray killed by reflect
[array([     0.,      0., -10000.]), array([ -4.62943511,  -2.42164114,  18.49164167])]
[ 0.  0.  0.]
[ 0.  0.  0.]
137 ray bounce number 1
[ -4.29042715   3.01233118  15.68865865]
x =  None
137 ray killed by reflect
[array([     0.,      0., -10000.]), array([ -4.29042715,   3.01233118,  15.68865865])]
[ 0.  0.  0.]
[ 0.  0.  0.]
138 ray bounce number 1
[  4.96777524  -1.5691008   20.83107445]
x =  None
138 ray killed by reflect
[array([     0.,      0., -10000.]), array([  4.96777524,  -1.5691008 ,  20.83107445])]
[ 0.  0.  0.]
[ 0.  0.  0.]
139 ray bounce number 1
[ -5.07510358  -0.87544103  30.04889256]
x =  None
139 ray killed by reflect
[array([     0.,      0., -10000.]), array([ -5.07510358,  -0.87544103,  30.04889256])]
[ 0.  0.

...

x =  [-0.02799379 -0.02736368  0.99923349]
862 ray bounce number 2
[  3.66540837   3.58290425  31.31145936]
x =  None
862 ray killed by reflect
[array([     0.,      0., -10000.]), array([  3.66540837,   3.58290425,  31.31145936]), array([  3.66540837,   3.58290425,  31.31145936])]
[ 0.  0.  0.]
[ 0.  0.  0.]
863 ray bounce number 1
[ -0.69403516   5.19558976  15.78013424]
x =  None
863 ray killed by reflect
[array([     0.,      0., -10000.]), array([ -0.69403516,   5.19558976,  15.78013424])]
[ 0.  0.  0.]
[ 0.  0.  0.]
864 ray bounce number 1
[ -1.99319171  -4.73524726  30.69143617]
x =  [ 0.01518365  0.03607197  0.99923384]
864 ray bounce number 2
[ -1.80977031  -4.29949105  42.7623705 ]
x =  None
864 ray killed by reflect
[array([     0.,      0., -10000.]), array([ -1.99319171,  -4.73524726,  30.69143617]), array([ -1.80977031,  -4.29949105,  42.7623705 ])]
[ 0.  0.  0.]
[ 0.  0.  0.]
865 ray bounce number 1
[ 5.27610094 -0.74682494  1.92043811]
x =  None
865 ray killed by reflec

x =  None
895 ray killed by reflect
[array([     0.,      0., -10000.]), array([  1.56008895,   5.01685442,  13.86711534])]
[ 0.  0.  0.]
[ 0.  0.  0.]
896 ray bounce number 1
[-1.20808031 -5.15354374  7.59787241]
x =  None
896 ray killed by reflect
[array([     0.,      0., -10000.]), array([-1.20808031, -5.15354374,  7.59787241])]
[ 0.  0.  0.]
[ 0.  0.  0.]
897 ray bounce number 1
[  2.32153172   4.73408147  10.87651988]
x =  None
897 ray killed by reflect
[array([     0.,      0., -10000.]), array([  2.32153172,   4.73408147,  10.87651988])]
[ 0.  0.  0.]
[ 0.  0.  0.]
898 ray bounce number 1
[ -5.23734413   0.26495595  15.41617004]
x =  None
898 ray killed by reflect
[array([     0.,      0., -10000.]), array([ -5.23734413,   0.26495595,  15.41617004])]
[ 0.  0.  0.]
[ 0.  0.  0.]
899 ray bounce number 1
[  3.52639353   3.92450277  10.33162573]
x =  None
899 ray killed by reflect
[array([     0.,      0., -10000.]), array([  3.52639353,   3.92450277,  10.33162573])]
[ 0.  0.  0.]


x =  None
929 ray killed by reflect
[array([     0.,      0., -10000.]), array([  2.67722306,  -4.41317137,  28.32920951])]
[ 0.  0.  0.]
[ 0.  0.  0.]
930 ray bounce number 1
[ -3.6747469   -3.70520891  19.45182122]
x =  None
930 ray killed by reflect
[array([     0.,      0., -10000.]), array([ -3.6747469 ,  -3.70520891,  19.45182122])]
[ 0.  0.  0.]
[ 0.  0.  0.]
931 ray bounce number 1
[ -0.9365492    5.1567373   15.88227381]
x =  None
931 ray killed by reflect
[array([     0.,      0., -10000.]), array([ -0.9365492 ,   5.1567373 ,  15.88227381])]
[ 0.  0.  0.]
[ 0.  0.  0.]
932 ray bounce number 1
[ -3.00233925  -4.15336428  31.35156952]
x =  None
932 ray killed by reflect
[array([     0.,      0., -10000.]), array([ -3.00233925,  -4.15336428,  31.35156952])]
[ 0.  0.  0.]
[ 0.  0.  0.]
933 ray bounce number 1
[  4.85542642  -1.79839131  25.82956437]
x =  None
933 ray killed by reflect
[array([     0.,      0., -10000.]), array([  4.85542642,  -1.79839131,  25.82956437])]
[ 0.  0.

x =  None
966 ray killed by reflect
[array([     0.,      0., -10000.]), array([ -4.46530295,   2.63903378,  24.41136136])]
[ 0.  0.  0.]
[ 0.  0.  0.]
967 ray bounce number 1
[ -3.37298183   3.86540281  31.07973545]
x =  None
967 ray killed by reflect
[array([     0.,      0., -10000.]), array([ -3.37298183,   3.86540281,  31.07973545])]
[ 0.  0.  0.]
[ 0.  0.  0.]
968 ray bounce number 1
[ -3.20428625  -3.99097932  31.70066991]
x =  [ 0.02451154  0.03052943  0.99923328]
968 ray bounce number 2
[ -2.9093954   -3.62368901  43.7221418 ]
x =  None
968 ray killed by reflect
[array([     0.,      0., -10000.]), array([ -3.20428625,  -3.99097932,  31.70066991]), array([ -2.9093954 ,  -3.62368901,  43.7221418 ])]
[ 0.  0.  0.]
[ 0.  0.  0.]
969 ray bounce number 1
[ 0.63098139  5.30300956  0.03469375]
x =  [-0.00153027 -0.01286098  0.99991612]
969 ray bounce number 2
[ 0.63098139  5.30300956  0.03469375]
x =  None
969 ray killed by reflect
[array([     0.,      0., -10000.]), array([ 0.63098

x =  None
998 ray killed by reflect
[array([     0.,      0., -10000.]), array([  0.48760651,  -5.15801735,  25.32403072])]
[ 0.  0.  0.]
[ 0.  0.  0.]
999 ray bounce number 1
[-5.31061899 -0.28777524  3.57156761]
x =  None
999 ray killed by reflect
[array([     0.,      0., -10000.]), array([-5.31061899, -0.28777524,  3.57156761])]
[ 0.  0.  0.]
         3135922 function calls in 353.070 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    34260    0.039    0.000    0.055    0.000 <frozen importlib._bootstrap>:402(parent)
    34260    0.041    0.000    0.059    0.000 <frozen importlib._bootstrap>:989(_handle_fromlist)
        1    0.000    0.000  353.069  353.069 <string>:1(<module>)
        3    0.000    0.000    0.000    0.000 _collections_abc.py:657(get)
     1116    0.002    0.000    0.312    0.000 _methods.py:25(_amax)
     1116    0.003    0.000    0.314    0.000 _methods.py:28(_amin)
    18184    0.011    0.000    0.

In [5]:
detector.catchRays(rays)

In [6]:
len(detector.rays)

53

## No energy response

In [10]:
from foxsisim.module import Module
from foxsisim.detector import Detector
from foxsisim.source import Source
from foxsisim.plotting import scatterHist
from foxsisim.plotting import plot
from foxsisim.util import save_rays
import matplotlib.pyplot as plt
from astropy.io import fits
import numpy as np

In [12]:
module = Module(radii = [5.151,4.9,4.659,4.429,4.21,4.0,3.799], #7Shells
    #module = Module(radii = [5.151,4.9,4.659,4.429,4.21,4.0,3.799,3.59,3.38,3.17], #10Shells
                    seglen=30.0,
                    base=[0,0,0],
                    focal=200,
                    angles=None,
                    conic=False)

detector = Detector(width=10, #27.6 x 27.6 mm values taken from Steven
                    height=10,
                    normal = [0,0,1],
                    center = [0,0,230],
                    reso = [2048,2048]) # Resolution taken from Steven

    #source_distance = -2187.5
source_distance = -1.04e4 #Source distance according to Steve's paper 
offaxis_angle_arcmin = 14.0
source = Source(type='point',
                center=[ source_distance * np.sin(np.deg2rad(offaxis_angle_arcmin/60.0)) , 0.0 , source_distance ],
                color=[1,1,1])
    
photons=100

In [13]:
cProfile.run('rays = source.generateRays(module.targetFront, nrays)')

         1913 function calls in 0.005 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.004    0.004 <string>:1(<module>)
      100    0.000    0.000    0.000    0.000 linalg.py:111(isComplexType)
      100    0.000    0.000    0.001    0.000 linalg.py:2022(norm)
        1    0.000    0.000    0.000    0.000 module.py:242(targetFront)
      100    0.000    0.000    0.000    0.000 numeric.py:463(asarray)
      100    0.000    0.000    0.001    0.000 ray.py:16(__init__)
        1    0.000    0.000    0.000    0.000 segmentp.py:253(targetFront)
        1    0.000    0.000    0.000    0.000 shell.py:75(targetFront)
        1    0.001    0.001    0.004    0.004 source.py:144(generateRays)
        1    0.000    0.000    0.000    0.000 source.py:159(<listcomp>)
        1    0.000    0.000    0.000    0.000 source.py:160(<listcomp>)
        1    0.000    0.000    0.001    0.001 source.py:170(<listcomp

In [14]:
cProfile.run('module.passRays(rays, robust=True)')

         632541 function calls in 0.957 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.957    0.957 <string>:1(<module>)
      100    0.000    0.000    0.002    0.000 circle.py:44(inRange)
     5234    0.002    0.000    0.002    0.000 fromnumeric.py:1549(shape)
      200    0.000    0.000    0.001    0.000 fromnumeric.py:504(transpose)
      200    0.000    0.000    0.000    0.000 fromnumeric.py:55(_wrapfunc)
     2617    0.004    0.000    0.005    0.000 getlimits.py:96(__new__)
      100    0.000    0.000    0.000    0.000 linalg.py:101(get_linalg_error_extobj)
      200    0.000    0.000    0.000    0.000 linalg.py:106(_makearray)
     3365    0.001    0.000    0.002    0.000 linalg.py:111(isComplexType)
      200    0.000    0.000    0.000    0.000 linalg.py:124(_realType)
      100    0.000    0.000    0.001    0.000 linalg.py:139(_commonType)
      100    0.000    0.000    0.000    0.0